In [1]:
import sys
sys.path.append('../AutoX')

In [2]:
from autox import AutoXRecommend

/home/caihengxing/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not mo

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [4]:
path = '~/data/客户数据_慕尚/autox_recommend_temp/'

In [5]:
inter_df = pd.read_csv(path + 'inter_df.csv')
user_df = pd.read_csv(path + 'user_df.csv')
item_df = pd.read_csv(path + 'item_df.csv')
test = pd.read_csv(path + 'test.csv')

In [6]:
item_df

,商品类型,price_max,price_min,price_median,price_mean,price_std,color_count
0,[01]单西,3442.00,0.0000,596.84,648.920190,331.100986,18394
1,[02]裤（长裤）,4050.00,-1.2900,291.00,326.717095,159.983925,972258
2,[03]衬衫（长袖）,36198.67,-0.2000,287.66,320.355620,175.394367,312622
3,[04]针织,599.00,599.0000,599.00,599.000000,NaN,1
4,[05]牛仔（长裤）,42998.67,-0.0129,317.77,345.009627,173.603361,401982
...,...,...,...,...,...,...,...
110,[U8]马甲,999.00,799.0000,939.00,919.000000,97.979590,4
111,[W0]运动文胸,33.93,33.9300,33.93,33.930000,NaN,1
112,[X3]吊带衫,269.00,268.9895,269.00,268.997375,0.005250,4
113,[YH]小饰品,NaN,NaN,NaN,NaN,NaN,10


In [7]:
uid = '会员id'
iid = '商品类型'
time_col = '订单时间'
recall_num = 100

In [8]:
inter_df[time_col].min(), inter_df[time_col].max()

('2020-01-01 00:00:43', '2022-04-06 23:59:05')

In [9]:
test[time_col].min(), test[time_col].max()

('2022-04-07 00:00:49', '2022-04-13 19:19:02')

In [10]:
positive_items_test = test.groupby([uid])[iid].apply(list)
test_users = positive_items_test.keys()
test_items = []

for i, user in tqdm(enumerate(test_users)):
    test_items.append(positive_items_test[user])
    
print("Total users in testidation:", len(test_users))

9037it [00:00, 194581.67it/s]

Total users in testidation: 9037


In [11]:
inter_df[time_col] = pd.to_datetime(inter_df[time_col])

In [12]:
autoXRecommend = AutoXRecommend()

autoXRecommend.fit(inter_df = inter_df, user_df = user_df, item_df = item_df,
                  uid = uid, iid = iid, time_col = time_col,
                  recall_num = recall_num,
                  debug = True, debug_save_path = './temp_ms6')


popular_recall
train
2022-03-24 00:00:06 2022-03-30 23:58:54


100%|██████████| 10722/10722 [00:00<00:00, 11868.07it/s]


HIT:  0.8462593591321527
valid
2022-03-31 00:00:07 2022-04-06 23:59:05


100%|██████████| 10393/10393 [00:00<00:00, 12518.84it/s]


HIT:  0.8464689376089789

history_recall
train
2022-03-24 00:00:06 2022-03-30 23:58:54


...


itemcf_recall
train
valid

binary_recall
train
valid

merge recalls
train
valid

feature engineer
train
valid
train_fe shape: (907101, 41)
valid_fe shape: (855009, 41)

ranker
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's map@12: 0.537345
[200]	valid_0's map@12: 0.537606
[300]	valid_0's map@12: 0.538347
[400]	valid_0's map@12: 0.538993
[500]	valid_0's map@12: 0.539925
[600]	valid_0's map@12: 0.540377
[700]	valid_0's map@12: 0.540593
[800]	valid_0's map@12: 0.540454
Early stopping, best iteration is:
[727]	valid_0's map@12: 0.540822
defaultdict(<class 'dict'>, {'valid_0': {'map@12': 0.5408222638227921}})
                        feature  importance
28  purchase_corr_item_max_time        2593
37                     商品类型_idx        2572
22                  price_max_y        1483
36     latest_purchase_time_sub        1068
29       purchase_corr_item_cnt        1064
27                  color_count        1028
23                  price_min_y         930
0  

100%|██████████| 10393/10393 [00:00<00:00, 11272.92it/s]


HIT:  0.8464689376089789

history_recall
2022-03-31 00:00:07 2022-04-06 23:59:05


100%|██████████| 5618/5618 [00:00<00:00, 23310.96it/s]



itemcf_recall

binary_recall

merge recalls

feature engineer
train_fe shape: (855009, 41)

ranker
[100]	valid_0's map@12: 0.536026
[200]	valid_0's map@12: 0.540378
[300]	valid_0's map@12: 0.543684
[400]	valid_0's map@12: 0.545884
[500]	valid_0's map@12: 0.55053
[600]	valid_0's map@12: 0.552374
[700]	valid_0's map@12: 0.55481
defaultdict(<class 'dict'>, {'valid_0': {'map@12': 0.5555553620505647}})
                        feature  importance
28  purchase_corr_item_max_time        2443
37                     商品类型_idx        2375
22                  price_max_y        1618
27                  color_count        1401
18                    首次注册的渠道类型        1233
23                  price_min_y         965
29       purchase_corr_item_cnt         947
36     latest_purchase_time_sub         940
0                  itemcf_score         798
21                    入会时间_diff         785
24               price_median_y         730
8                price_median_x         658
6                  price_m

In [13]:
res = autoXRecommend.transform(test_users)


popular recall, test
2022-03-31 00:00:07 2022-04-06 23:59:05


100%|██████████| 9037/9037 [00:00<00:00, 14281.47it/s]



history recall, test

itemcf recall, test

binary recall, test

merge recalls

feature engineer
customer feature engineer
num_cols: ['price', 'sales_channel_id']
cat_cols: []
interact feature engineer
test_fe shape: (742298, 40)

inference
[1/1]
(742298, 40)


In [14]:
def apk(actual, predicted, k=12):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [15]:
outputs = res['prediction']
print("mAP Score on Validation set:", mapk(test_items, outputs))

mAP Score on Validation set: 0.4718946600598702


In [80]:
# res.to_csv('./recall_and_rank.csv', index = False)

In [61]:
rank_dict = res.set_index('会员id').T.to_dict('list')

In [64]:
import pickle
def save_obj(obj, name):
    with open(f'{name}.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [65]:
save_obj(rank_dict, 'recall_and_rank')

In [66]:
def load_obj(name):
    with open(f'{name}.pkl', 'rb') as f:
        return pickle.load(f)

In [68]:
recall_and_rank = load_obj('recall_and_rank')

In [23]:
uid = '00f73578e4cc11e8840998039b46e950'

In [76]:
popular_item = ['[44]',
                 '[02]',
                 '[21]',
                 '[24]',
                 '[05]',
                 '[27]',
                 '[31]',
                 '[03]',
                 '[23]',
                 '[99]',
                 '[22]',
                 '[74]']

In [77]:
if uid in recall_and_rank:
    cur_res = recall_and_rank[uid][0][:12]
    cur_res = [x[:4] for x in cur_res]
else:
    cur_res = popular_item

In [79]:
cur_res

['[44]',
 '[02]',
 '[21]',
 '[05]',
 '[31]',
 '[03]',
 '[24]',
 '[27]',
 '[23]',
 '[22]',
 '[50]',
 '[34]']

In [83]:
res

,会员id,prediction
0,00aa3029e4cd11e8840998039b46e950,"[[44]短袖针织, [02]裤（长裤）, [21]夹克外套, [05]牛仔（长裤）, [2..."
1,00f73578e4cc11e8840998039b46e950,"[[44]短袖针织, [02]裤（长裤）, [21]夹克外套, [05]牛仔（长裤）, [3..."
2,0156a90de4ce11e8840998039b46e950,"[[44]短袖针织, [02]裤（长裤）, [21]夹克外套, [05]牛仔（长裤）, [2..."
3,016c92a4e4ce11e8840998039b46e950,"[[44]短袖针织, [02]裤（长裤）, [21]夹克外套, [99]物料, [24]PO..."
4,0174f217e4ca11e8840998039b46e950,"[[44]短袖针织, [02]裤（长裤）, [21]夹克外套, [05]牛仔（长裤）, [3..."
...,...,...
9032,ff15e7a6e4ca11e8840998039b46e950,"[[44]短袖针织, [02]裤（长裤）, [21]夹克外套, [05]牛仔（长裤）, [2..."
9033,ff20eca5e4ca11e8840998039b46e950,"[[44]短袖针织, [02]裤（长裤）, [21]夹克外套, [05]牛仔（长裤）, [2..."
9034,ff840c42e4c811e8840998039b46e950,"[[44]短袖针织, [02]裤（长裤）, [21]夹克外套, [05]牛仔（长裤）, [3..."
9035,ffb64c3de4c811e8840998039b46e950,"[[44]短袖针织, [02]裤（长裤）, [21]夹克外套, [05]牛仔（长裤）, [3..."


In [81]:
# 终端执行:
#     python app.py

# 终端调用:

# curl -X GET http://127.0.0.1:5000/ -d uid='016c92a4e4ce11e8840998039b46e950'

# curl -X GET http://127.0.0.1:5000/ -d uid='00f73578e4cc11e8840998039b46e950'